# Model Playground

In [2]:
from pprint import pprint
import numpy as np

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

## Layer Class example

In [ ]:
class Encoder(layers.Layer):
  """Maps MNIST digits to a triplet (z_mean, z_log_var, z)."""

  def __init__(self,
               latent_dim=32,
               intermediate_dim=64,
               name='encoder',
               **kwargs):
    super(Encoder, self).__init__(name=name, **kwargs)
    self.dense_proj = layers.Dense(intermediate_dim, activation='relu')
    self.dense_mean = layers.Dense(latent_dim)
    self.dense_log_var = layers.Dense(latent_dim)
    self.sampling = Sampling()

  def call(self, inputs):
    x = self.dense_proj(inputs)
    z_mean = self.dense_mean(x)
    z_log_var = self.dense_log_var(x)
    z = self.sampling((z_mean, z_log_var))
    return z_mean, z_log_var, z

## ConvStack Class

In [125]:
class ConvStack(layers.Layer):
  """Creates a Conv2D + BatchNorm stack.
  If input is (1,21,21), output is (1, 1, 64, 3, 3)"""

  def __init__(self,
               input_shape:tuple = (1,21,21),
               weight_decay=1e-5,
               filters=[32, 64, 64],
               kernel_sizes = [(5,5), (3,3), (3,3)],
               strides=[(2,2),(1,1),(2,2)],
               bias_init=0.1,
               output_activation=tf.nn.sigmoid,
               name='encoder',
               **kwargs):
    super(ConvStack, self).__init__(name=name, **kwargs)

    self.Conv2D_1 = layers.Conv2D(name="conv1",
                                  input_shape = input_shape,
                                 data_format='channels_first',
                                 filters=filters[0],
                                 kernel_size=kernel_sizes[0],
                                 strides=strides[0],
                                 kernel_initializer=tf.keras.initializers.GlorotNormal(),
                                 activity_regularizer=tf.keras.regularizers.l2(l=weight_decay),
                                 activation=tf.nn.relu,
                                 )
    self.BN_1 = layers.BatchNormalization(axis=1, name="conv1_bn")
    self.Conv2D_2 = layers.Conv2D(name="conv2",
                                 data_format='channels_first',
                                 filters=filters[1],
                                 kernel_size=kernel_sizes[1],
                                 strides=strides[1],
                                 kernel_initializer=tf.keras.initializers.GlorotNormal(),
                                 activity_regularizer=tf.keras.regularizers.l2(l=weight_decay),
                                 activation=tf.nn.relu,
                                 )
    self.BN_2 = layers.BatchNormalization(axis=1, name="conv2_bn")
    self.Conv2D_3 = layers.Conv2D(name="conv3",
                                 data_format='channels_first',
                                 filters=filters[2],
                                 kernel_size=kernel_sizes[2],
                                 strides=strides[2],
                                 kernel_initializer=tf.keras.initializers.GlorotNormal(),
                                 activity_regularizer=tf.keras.regularizers.l2(l=weight_decay),
                                 activation=tf.nn.relu,
                                 )
    self.BN_3 = layers.BatchNormalization(axis=1, name="conv3_bn")
    

  def call(self, inputs):
    """Construct the stack and pass the inputs thru"""
    stack = self.Conv2D_1(inputs)
    stack = self.BN_1(stack)
    stack = self.Conv2D_2(stack)
    stack = self.BN_2(stack)
    stack = self.Conv2D_3(stack)
    stack = self.BN_3(stack)
    # Add an empty dim to pass to Encoder
    return tf.expand_dims(stack, 0)

In [132]:
inputs = tf.Variable(tf.random.uniform([1,1,21,21], -1, 1))
convstack = ConvStack()
convstack.call(inputs)
# stack.call(input for inputs in tf.unstack(inputs,num=24, axis=1))

<tf.Tensor: shape=(1, 1, 64, 3, 3), dtype=float32, numpy=
array([[[[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
          [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
          [1.54450592e-02, 0.00000000e+00, 0.00000000e+00]],

         [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
          [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
          [0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

         [[1.18545834e-02, 0.00000000e+00, 0.00000000e+00],
          [2.62208506e-02, 2.73919627e-02, 0.00000000e+00],
          [0.00000000e+00, 6.35615364e-02, 0.00000000e+00]],

         [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
          [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
          [0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

         [[1.18993811e-01, 3.44521329e-02, 0.00000000e+00],
          [6.33240268e-02, 6.98077232e-02, 8.45875684e-03],
          [8.84230435e-02, 4.33336906e-02, 3.68882082e-02]],

         [[8.09396431e-02, 0.000

## Encoder Class

In [57]:
class Encoder(tf.keras.Model):
    def __init__(self, name='Encoder_Stack', cells=24, cell_input_shape=(1, 1, 64, 3, 3), lstm_layers=1, 
                 lstm_ksize_input=(3, 3), lstm_ksize_hidden=(5, 5),
                 lstm_use_peepholes=True, lstm_cell_clip=None, 
                 lstm_bn_input_hidden=False, 
                 lstm_bn_hidden_hidden=False,
                 lstm_bn_peepholes=False,):
        
        super(Encoder, self).__init__(name=name)
        # create a list of InputLayers
        self.input_layers = []
        # create a list of ConvLSTM cells
        self.encoder_cells = []
        self.encoder_states = ['barf']
        
        
        i = 0
        # First Frame
        self.encoder_cells.append(layers.ConvLSTM2D(input_shape = cell_input_shape,
                                   name="encoderConvLSTM{}".format(i+1),
                               filters = lstm_layers,
                               kernel_size=lstm_ksize_input,
                               padding='same',
                               data_format='channels_first',
                               return_sequences=True,
                               return_state=True)
        )
        
        # for each frame, create an InputLayer and corresponding ConvLSTM cell
        for i in range(1, cells):
            self.input_layers.append(
                layers.Input(name="encoder{}_input".format(i+1),
                          shape=cell_input_shape)
            )
            
            self.encoder_cells.append(layers.ConvLSTM2D(name="encoderConvLSTM{}".format(i+1),
                                   filters = lstm_layers,
                                   kernel_size=lstm_ksize_hidden,
                                   padding='same',
                                   data_format='channels_first',
                                   return_sequences=True,
                                   return_state=True)
            )
        
    # input to call() will be a list of tensors of shape=(1, 1, 64, 3, 3)
    # call() will take list of inputs and assign each to InputLayer
    # call returns
    def call(self, inputs_list):
#             assert len(inputs_list) == len(self.input_layers), "Input list length does not match # input layers in stack"
        assert len(inputs_list) == len(self.encoder_cells), "Input list length does not match # encoder cells in stack"

        # connect each input to its corresponding InputLayer,
        # and connect encoder hidden & cell states 

#             current_input = self.input_layers[0](inputs_list[0])
        _, self.state_h, self.state_c = self.encoder_cells[0](inputs_list[0])
        print(i, self.state_h.shape, self.state_c.shape)
        self.encoder_states = [self.state_h, self.state_c]

        for i in range(1, len(inputs_list)):
#                 current_input = self.input_layers[i](inputs_list[i])
            _, self.state_h, self.state_c = self.encoder_cells[i](inputs_list[i], initial_state = self.encoder_states)
            print(i, self.state_h.shape, self.state_c.shape)
            self.encoder_states = [self.state_h, self.state_c]
        return self.encoder_states

## test Encoder class

In [58]:
enco.encoder_cells[0].get_config()

{'name': 'encoderConvLSTM1',
 'trainable': True,
 'dtype': 'float32',
 'return_sequences': True,
 'return_state': True,
 'go_backwards': False,
 'stateful': False,
 'unroll': False,
 'time_major': False,
 'filters': 1,
 'kernel_size': (3, 3),
 'strides': (1, 1),
 'padding': 'same',
 'data_format': 'channels_first',
 'dilation_rate': (1, 1),
 'activation': 'tanh',
 'recurrent_activation': 'hard_sigmoid',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'GlorotUniform',
  'config': {'seed': None}},
 'recurrent_initializer': {'class_name': 'Orthogonal',
  'config': {'gain': 1.0, 'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'unit_forget_bias': True,
 'kernel_regularizer': None,
 'recurrent_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'recurrent_constraint': None,
 'bias_constraint': None,
 'dropout': 0.0,
 'recurrent_dropout': 0.0}

In [61]:
enco_model = Encoder(cells=1)
inputs = [tf.Variable(tf.random.uniform([1, 1, 64, 3, 3], -1, 1))]
enco_model.compile(loss='mean_squared_error',
                  optimizer='adadelta',
                  metrics=['accuracy', 'mean_absolute_error'])
enco_model.build((1,1,64,3,3))
enco_model.summary()
# x = enco.call(inputs)
# len(x)

ValueError: You cannot build your model by calling `build` if your layers do not support float type inputs. Instead, in order to instantiate and build your model, `call` your model on real tensor data (of the correct dtype).

## Decoder Class

In [165]:
class Decoder(layers.Layer):
    def __init__(self, cells=24, cell_input_shape=(1, 1, 64, 3, 3), lstm_layers=1, 
                 lstm_ksize_input=(3, 3), lstm_ksize_hidden=(5, 5),
                 lstm_use_peepholes=True, lstm_cell_clip=None, 
                 lstm_bn_input_hidden=False, 
                 lstm_bn_hidden_hidden=False,
                 lstm_bn_peepholes=False,):
        # create a list of InputLayers
        self.input_layers = []
        # create a list of ConvLSTM cells
        self.decoder_cells = []
        # Decoder sends output
        self.decoder_outputs = []
        
        # First Frame
        self.input_layers.append(
            layers.Input(name="decoder{}_input".format(0),
                      shape=cell_input_shape)
        )

        self.decoder_cells.append(layers.ConvLSTM2D(name="decoder{}".format(0),
                               filters = lstm_layers,
                               kernel_size=lstm_ksize_input,
                               padding='same',
                               data_format='channels_first',
                               return_sequences=True,
                               return_state=True)
        )
        
        # for each frame, create an InputLayer and corresponding ConvLSTM cell
        for i in range(1, cells):
            self.input_layers.append(
                layers.Input(name="decoder{}_input".format(i+1),
                          shape=cell_input_shape)
            )
            
            self.decoder_cells.append(layers.ConvLSTM2D(name="decoder{}".format(i+1),
                                   filters = lstm_layers,
                                   kernel_size=lstm_ksize_hidden,
                                   padding='same',
                                   data_format='channels_first',
                                   return_sequences=True,
                                   return_state=True)
            )
        
    # input to call() will be a list of tensors of shape=(1, 1, 64, 3, 3)
    # call() will take list of inputs and assign each to InputLayer
    def call(self, inputs_list, encoder_states):
        assert len(inputs_list) == len(self.input_layers), "Input list length does not match # input layers in stack"
        assert len(inputs_list) == len(self.decoder_cells), "Input list length does not match # encoder cells in stack"

        # initialize first decoder cell with encoder states
        current_input = self.input_layers[0](inputs_list[0])
        decoder_output, state_h, state_c = self.decoder_cells[0](current_input, initial_state = encoder_states)
        decoder_states = [state_h, state_c]
        self.decoder_outputs.append(decoder_output)

        for i in range(1, len(inputs_list)):
            current_input = self.input_layers[i](inputs_list[i])
            _, state_h, state_c = self.decoder_cells[i](current_input, initial_state = decoder_states)
            decoder_states = [state_h, state_c]
        return self.decoder_outputs

In [166]:
deco = Decoder()
inputs = [tf.Variable(tf.random.uniform([1, 1, 64, 3, 3], -1, 1))]
deco.call(inputs, [enco_states])

TypeError: call() takes 2 positional arguments but 3 were given

In [48]:
class Encoder_1f(tf.keras.Model):
    def __init__(self, name='Encoder_Stack', cells=1, cell_input_shape=(1, 1, 64, 3, 3), lstm_layers=1, 
                 lstm_ksize_input=(3, 3), lstm_ksize_hidden=(5, 5),
                 lstm_use_peepholes=True, lstm_cell_clip=None, 
                 lstm_bn_input_hidden=False, 
                 lstm_bn_hidden_hidden=False,
                 lstm_bn_peepholes=False,):
        
        super(Encoder_1f, self).__init__(name=name)
        self.cell_input_shape=cell_input_shape

        # First Frame
        self.encoder_cells = layers.ConvLSTM2D(input_shape = cell_input_shape,
                                name="encoderConvLSTM{}".format(1),
                                filters = lstm_layers,
                                kernel_size=lstm_ksize_input,
                                padding='same',
                                data_format='channels_first',
                                return_sequences=True,
                                return_state=True)
        
        

        # input to call() will be a list of tensors of shape=(1, 1, 64, 3, 3)
        # call() will take list of inputs and assign each to InputLayer
        # call returns
    def call(self, inputs_list):
#             assert len(inputs_list) == len(self.input_layers), "Input list length does not match # input layers in stack"
        assert len(inputs_list) == len(self.encoder_cells), "Input list length does not match # encoder cells in stack"

        # connect each input to its corresponding InputLayer,
        # and connect encoder hidden & cell states 

#             current_input = self.input_layers[0](inputs_list[0])
        _, self.state_h, self.state_c = self.encoder_cells(inputs_list[0])
        print(i, self.state_h.shape, self.state_c.shape)
        self.encoder_states = [self.state_h, self.state_c]

        return self.encoder_states

In [49]:
enco_1f = Encoder_1f()

In [50]:
enco_1f.cell_input_shape

(1, 1, 64, 3, 3)

In [56]:
enco_1f(tf.Variable(tf.random.uniform([1, 1, 64, 3, 3], -1, 1)))

TypeError: object of type 'ResourceVariable' has no len()

In [53]:

enco_1f.compile(loss='mean_squared_error',
                  optimizer='adadelta',
                  metrics=['accuracy', 'mean_absolute_error'])
enco_1f.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.